In [1]:
import csv
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

plt.style.use('bmh')

In [2]:
launches = pd.read_csv('data/launches_20201027.csv')
notam = pd.read_csv('data/notam_20201027_commas_quotes.csv', on_bad_lines='skip', encoding='utf-16', low_memory=False)
polygon = pd.read_csv('data/polygon_20201027.csv')
spaceports = pd.read_csv('data/spaceports_20201027.csv')
vertices = pd.read_csv('data/vertices_20201027.csv')

EDA on Launches

In [4]:
launches.head(5)

,LAUNCHES_REC_ID,LAUNCH_DATE,VEHICLE_NAME,PAD_NAME,PAYLOAD,AGENCY,PURPOSE,ORBIT_TYPE,OUTCOME,SPACE_LAUNCH_REPORT_FLAG,GUNTER_FLAG,SPACE_FLIGHT_NOW_FLAG,NOONAN_FLAG,AST_FLAG,WIKIPEDIA_FLAG,JSR_FLAG,SPACEPORT_REC_ID
0,1,2013-01-05 00:00:00,STIG-B III,NaN,Scientific,Armadillo Aerospace,NaN,NaN,Failure,False,False,False,False,True,False,True,19.0
1,2,2013-01-26 22:00:00,Ground Based Interceptor,LF-23,NaN,Missile Defense Agency,Anti-ballistic missile test,Sub-orbital,Successful,False,False,False,False,False,True,True,23.0
2,3,2013-01-26 22:50:00,Terrier Improved Orion,LA-2,NaN,NASA,Atmospheric Experiments,Sub-orbital,Successful,False,False,False,False,False,True,True,9.0
3,4,2013-01-31 01:48:00,Atlas 5-401,SLC-41,TDRS-K,United Launch Alliance,Communications,Geosynchronous,Operational,True,True,True,False,False,True,True,2.0
4,5,2013-02-01 06:56:00,Zenit 3SL/DMSL,Odyssey,Intelsat 27,"Energia Logistics, Ltd. (ELUS)",Communications,NaN,Failure,True,True,True,False,True,True,False,NaN


In [5]:
launches.shape

(518, 17)

In [6]:
# Data cleaning: Remove all leading and trailing spaces in the data
launches.columns = launches.columns.str.strip()
launches

,LAUNCHES_REC_ID,LAUNCH_DATE,VEHICLE_NAME,PAD_NAME,PAYLOAD,AGENCY,PURPOSE,ORBIT_TYPE,OUTCOME,SPACE_LAUNCH_REPORT_FLAG,GUNTER_FLAG,SPACE_FLIGHT_NOW_FLAG,NOONAN_FLAG,AST_FLAG,WIKIPEDIA_FLAG,JSR_FLAG,SPACEPORT_REC_ID
0,1,2013-01-05 00:00:00,STIG-B III,NaN,Scientific,Armadillo Aerospace,NaN,NaN,Failure,False,False,False,False,True,False,True,19.0
1,2,2013-01-26 22:00:00,Ground Based Interceptor,LF-23,NaN,Missile Defense Agency,Anti-ballistic missile test,Sub-orbital,Successful,False,False,False,False,False,True,True,23.0
2,3,2013-01-26 22:50:00,Terrier Improved Orion,LA-2,NaN,NASA,Atmospheric Experiments,Sub-orbital,Successful,False,False,False,False,False,True,True,9.0
3,4,2013-01-31 01:48:00,Atlas 5-401,SLC-41,TDRS-K,United Launch Alliance,Communications,Geosynchronous,Operational,True,True,True,False,False,True,True,2.0
4,5,2013-02-01 06:56:00,Zenit 3SL/DMSL,Odyssey,Intelsat 27,"Energia Logistics, Ltd. (ELUS)",Communications,NaN,Failure,True,True,True,False,True,True,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
513,514,2020-04-22 19:30:30,Falcon 9 v1.2,LC-39A,Starlink 1 F6,SpaceX,Communications,Low Earth,Operational,True,True,True,False,True,True,False,8.0
514,515,2020-05-17 13:14:00,Atlas 5-501,SLC-41,USSF 7 (X37B OTV6),US Military,Testing,Low Earth,Operational,True,True,True,False,False,True,False,2.0
515,516,2020-05-25 18:50:00,LauncherOne,NaN,Starshine 4/Flight 1,Virgin Orbit,Testing,Low Earth,Failure,False,True,True,False,True,True,False,11.0
516,517,2020-05-30 19:22:45,Falcon 9 v1.2,LC-39A,DM-2/Crew Dragon 2,SpaceX,Testing,Low Earth,Operational,True,True,True,False,False,True,False,8.0


In [7]:
duplicates = launches.duplicated()
print(f'Number of duplicate rows = {duplicates}')

Number of duplicate rows = 0      False
1      False
2      False
3      False
4      False
       ...  
513    False
514    False
515    False
516    False
517    False
Length: 518, dtype: bool


In [8]:
launches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518 entries, 0 to 517
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   LAUNCHES_REC_ID           518 non-null    int64  
 1   LAUNCH_DATE               518 non-null    object 
 2   VEHICLE_NAME              518 non-null    object 
 3   PAD_NAME                  302 non-null    object 
 4   PAYLOAD                   240 non-null    object 
 5   AGENCY                    516 non-null    object 
 6   PURPOSE                   284 non-null    object 
 7   ORBIT_TYPE                267 non-null    object 
 8   OUTCOME                   485 non-null    object 
 9   SPACE_LAUNCH_REPORT_FLAG  518 non-null    bool   
 10  GUNTER_FLAG               518 non-null    bool   
 11  SPACE_FLIGHT_NOW_FLAG     518 non-null    bool   
 12  NOONAN_FLAG               518 non-null    bool   
 13  AST_FLAG                  518 non-null    bool   
 14  WIKIPEDIA_

In [9]:
## Probably not very useful in this dataset since the numeric columns are only the data/record IDs
launches.describe().round(3)

,LAUNCHES_REC_ID,SPACEPORT_REC_ID
count,518.000,456.000
mean,259.500,12.303
std,149.678,8.222
min,1.000,2.000
25%,130.250,2.000
50%,259.500,11.000
75%,388.750,23.000
max,518.000,24.000


In [10]:
launches.dtypes

LAUNCHES_REC_ID               int64
LAUNCH_DATE                  object
VEHICLE_NAME                 object
PAD_NAME                     object
PAYLOAD                      object
AGENCY                       object
PURPOSE                      object
ORBIT_TYPE                   object
OUTCOME                      object
SPACE_LAUNCH_REPORT_FLAG       bool
GUNTER_FLAG                    bool
SPACE_FLIGHT_NOW_FLAG          bool
NOONAN_FLAG                    bool
AST_FLAG                       bool
WIKIPEDIA_FLAG                 bool
JSR_FLAG                       bool
SPACEPORT_REC_ID            float64
dtype: object

In [11]:
## What columns include null values; and how many??
launches.isnull().sum()

LAUNCHES_REC_ID               0
LAUNCH_DATE                   0
VEHICLE_NAME                  0
PAD_NAME                    216
PAYLOAD                     278
AGENCY                        2
PURPOSE                     234
ORBIT_TYPE                  251
OUTCOME                      33
SPACE_LAUNCH_REPORT_FLAG      0
GUNTER_FLAG                   0
SPACE_FLIGHT_NOW_FLAG         0
NOONAN_FLAG                   0
AST_FLAG                      0
WIKIPEDIA_FLAG                0
JSR_FLAG                      0
SPACEPORT_REC_ID             62
dtype: int64

EDA on Polygon

In [12]:
polygon.head(5)

,POLYGON_ID,NOTAM_REC_ID,SUA_NAME,SUA_TYPE
0,1,1,VOLK SOUTH,MOA
1,2,2,VOLK EAST,MOA
2,3,3,FALLS 1,MOA
3,4,4,FALLS 2,MOA
4,5,5,VOLK WEST,MOA


In [13]:
polygon.shape

(1261928, 4)

In [14]:
# Data cleaning: Remove all leading and trailing spaces in the data
polygon.columns = polygon.columns.str.strip()
polygon

,POLYGON_ID,NOTAM_REC_ID,SUA_NAME,SUA_TYPE
0,1,1,VOLK SOUTH,MOA
1,2,2,VOLK EAST,MOA
2,3,3,FALLS 1,MOA
3,4,4,FALLS 2,MOA
4,5,5,VOLK WEST,MOA
...,...,...,...,...
1261923,1261948,1650464,NaN,NaN
1261924,1261949,1650465,NaN,NaN
1261925,1261950,1650466,NaN,NaN
1261926,1261951,1650467,NaN,NaN


In [15]:
duplicates = polygon.duplicated()
print(f'Number of duplicate rows = {duplicates}')

Number of duplicate rows = 0          False
1          False
2          False
3          False
4          False
           ...  
1261923    False
1261924    False
1261925    False
1261926    False
1261927    False
Length: 1261928, dtype: bool


In [16]:
polygon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1261928 entries, 0 to 1261927
Data columns (total 4 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   POLYGON_ID    1261928 non-null  int64 
 1   NOTAM_REC_ID  1261928 non-null  int64 
 2   SUA_NAME      227188 non-null   object
 3   SUA_TYPE      227188 non-null   object
dtypes: int64(2), object(2)
memory usage: 38.5+ MB


In [17]:
## Probably not very useful in this dataset since the numeric columns are only the data/record IDs
polygon.describe().round(3)

,POLYGON_ID,NOTAM_REC_ID
count,1261928.000,1261928.000
mean,630978.518,845527.117
std,364294.186,477888.221
min,1.000,1.000
25%,315487.750,434032.750
50%,630981.500,858338.500
75%,946464.250,1262659.250
max,1261952.000,1650468.000


In [18]:
polygon.dtypes

POLYGON_ID       int64
NOTAM_REC_ID     int64
SUA_NAME        object
SUA_TYPE        object
dtype: object

In [19]:
## What columns include null values; and how many??
polygon.isnull().sum()

POLYGON_ID            0
NOTAM_REC_ID          0
SUA_NAME        1034740
SUA_TYPE        1034740
dtype: int64

EDA on Spaceports

In [20]:
spaceports.head(5)

,SPACEPORT_REC_ID,SPACEPORT_NAME,LATITUDE,LONGITUDE,OWNER_NAME,FAA_LOC_ID,LOCATION_1,LOCATION_2
0,1,Blue Origin Landing Platform Strip,NaN,NaN,Blue Origin,NaN,NaN,NaN
1,2,Cape Canaveral Air Force Station,28.488889,-80.577778,Department of Defense,XMR,Cape Canaveral,Florida
2,3,Cecil Spaceport,30.218611,-81.876667,Jacksonville Aviation Authority,VQQ,Jacksonville,Florida
3,4,Colorado Air and Space Port,39.785278,-104.543056,Front Range Airport Authority,CFO,Adams County,Colorado
4,5,Corn Ranch,31.423333,-104.758889,Blue Origin,NaN,Van Horn,Texas


In [21]:
spaceports.shape

(24, 8)

In [22]:
# Data cleaning: Remove all leading and trailing spaces in the data
spaceports.columns = spaceports.columns.str.strip()
spaceports

,SPACEPORT_REC_ID,SPACEPORT_NAME,LATITUDE,LONGITUDE,OWNER_NAME,FAA_LOC_ID,LOCATION_1,LOCATION_2
0,1,Blue Origin Landing Platform Strip,NaN,NaN,Blue Origin,NaN,NaN,NaN
1,2,Cape Canaveral Air Force Station,28.488889,-80.577778,Department of Defense,XMR,Cape Canaveral,Florida
2,3,Cecil Spaceport,30.218611,-81.876667,Jacksonville Aviation Authority,VQQ,Jacksonville,Florida
3,4,Colorado Air and Space Port,39.785278,-104.543056,Front Range Airport Authority,CFO,Adams County,Colorado
4,5,Corn Ranch,31.423333,-104.758889,Blue Origin,NaN,Van Horn,Texas
5,6,Edwards Air Force Base,34.905556,-117.883611,Department of Defense,EDW,Lancaster,California
6,7,Ellington Airport,29.607222,-95.158889,City of Houston,EFD,Houston,Texas
7,8,John F. Kennedy Space Center,28.524167,-80.650833,NASA,NaN,Merritt Island,Florida
8,9,Mid-Atlantic Regional Spaceport,37.843333,-75.478056,NASA,NaN,Wallops Island,Virginia
9,10,Midland International Air and Space Port,31.942500,-102.201944,City of Midland,MAF,Midland,Texas


In [23]:
duplicates = spaceports.duplicated()
print(f'Number of duplicate rows = {duplicates}')

Number of duplicate rows = 0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
dtype: bool


In [24]:
spaceports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   SPACEPORT_REC_ID  24 non-null     int64  
 1   SPACEPORT_NAME    24 non-null     object 
 2   LATITUDE          23 non-null     float64
 3   LONGITUDE         23 non-null     float64
 4   OWNER_NAME        23 non-null     object 
 5   FAA_LOC_ID        12 non-null     object 
 6   LOCATION_1        23 non-null     object 
 7   LOCATION_2        23 non-null     object 
dtypes: float64(2), int64(1), object(5)
memory usage: 1.6+ KB


In [25]:
## Again, probably not very useful since just ID and geocoordinates here...
spaceports.describe().round(3)

,SPACEPORT_REC_ID,LATITUDE,LONGITUDE
count,24.000,23.000,23.000
mean,12.500,32.778,-81.602
std,7.071,11.116,81.761
min,1.000,9.397,-159.785
25%,6.750,28.519,-112.427
50%,12.500,31.423,-99.201
75%,18.250,34.982,-81.157
max,24.000,65.117,167.472


In [26]:
spaceports.dtypes

SPACEPORT_REC_ID      int64
SPACEPORT_NAME       object
LATITUDE            float64
LONGITUDE           float64
OWNER_NAME           object
FAA_LOC_ID           object
LOCATION_1           object
LOCATION_2           object
dtype: object

In [27]:
## What columns include null values; and how many??
spaceports.isnull().sum()

SPACEPORT_REC_ID     0
SPACEPORT_NAME       0
LATITUDE             1
LONGITUDE            1
OWNER_NAME           1
FAA_LOC_ID          12
LOCATION_1           1
LOCATION_2           1
dtype: int64

EDA on Vertices

In [28]:
vertices.head(5)

,VERTICES_REC_ID,POLYGON_ID,LATITUDE,LONGITUDE,VERTEX_NUM
0,1,1,44.166667,-90.183333,1
1,2,1,44.166667,-89.983333,2
2,3,1,43.666667,-89.770833,3
3,4,1,44.000000,-90.433333,4
4,5,1,44.000000,-90.587500,5


In [29]:
vertices.shape

(41006644, 5)

In [30]:
# Data cleaning: Remove all leading and trailing spaces in the data
vertices.columns = vertices.columns.str.strip()
vertices

,VERTICES_REC_ID,POLYGON_ID,LATITUDE,LONGITUDE,VERTEX_NUM
0,1,1,44.166667,-90.183333,1
1,2,1,44.166667,-89.983333,2
2,3,1,43.666667,-89.770833,3
3,4,1,44.000000,-90.433333,4
4,5,1,44.000000,-90.587500,5
...,...,...,...,...,...
41006639,41006664,1261952,-30.262954,153.042953,32
41006640,41006665,1261952,-30.252664,153.054820,33
41006641,41006666,1261952,-30.244318,153.068562,34
41006642,41006667,1261952,-30.238170,153.083763,35


In [31]:
duplicates = vertices.duplicated()
print(f'Number of duplicate rows = {duplicates}')

Number of duplicate rows = 0           False
1           False
2           False
3           False
4           False
            ...  
41006639    False
41006640    False
41006641    False
41006642    False
41006643    False
Length: 41006644, dtype: bool


In [32]:
vertices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41006644 entries, 0 to 41006643
Data columns (total 5 columns):
 #   Column           Dtype  
---  ------           -----  
 0   VERTICES_REC_ID  int64  
 1   POLYGON_ID       int64  
 2   LATITUDE         float64
 3   LONGITUDE        float64
 4   VERTEX_NUM       int64  
dtypes: float64(2), int64(3)
memory usage: 1.5 GB


In [33]:
## Probably not very useful in this dataset 
vertices.describe().round(1)

,VERTICES_REC_ID,POLYGON_ID,LATITUDE,LONGITUDE,VERTEX_NUM
count,41006644.0,41006644.0,4.100664e+07,41006644.0,41006644.0
mean,20503336.3,623745.6,-6.018293e+17,-39.6,66.5
std,11837605.5,366676.2,7.865634e+20,83.0,116.0
min,1.0,1.0,-1.370863e+24,-3664.7,1.0
25%,10251666.8,306955.0,3.290000e+01,-105.6,11.0
50%,20503339.5,626041.0,4.330000e+01,-80.6,25.0
75%,30755001.2,937647.0,4.680000e+01,20.7,60.0
max,41006668.0,1261952.0,1.578221e+13,3537.5,1000.0


In [34]:
vertices.dtypes

VERTICES_REC_ID      int64
POLYGON_ID           int64
LATITUDE           float64
LONGITUDE          float64
VERTEX_NUM           int64
dtype: object

In [35]:
## What columns include null values; and how many??
vertices.isnull().sum()

VERTICES_REC_ID    0
POLYGON_ID         0
LATITUDE           0
LONGITUDE          0
VERTEX_NUM         0
dtype: int64

NOTAM EDA

In [37]:
notam.shape

(1649078, 37)

In [38]:
notam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1649078 entries, 0 to 1649077
Data columns (total 37 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   NOTAM_REC_ID            1649078 non-null  int64  
 1   FNS_ID                  1649078 non-null  object 
 2   FILENAME                1649078 non-null  object 
 3   NOTAM_ID                1649049 non-null  object 
 4   NOTAM_TYPE              1427935 non-null  object 
 5   RELATED_NOTAM_ID        368132 non-null   object 
 6   SIMPLE_TEXT             827962 non-null   object 
 7   LOC_ID_ACCOUNTABLE_ORG  785649 non-null   object 
 8   NOTAM_NUMBER            782486 non-null   object 
 9   RELATED_NOTAM_NUMBER    125566 non-null   object 
 10  TEXT                    1641012 non-null  object 
 11  Q_CODE                  1008959 non-null  object 
 12  Q_CODE_INTERPRETATION   740483 non-null   object 
 13  A_CODE                  1141035 non-null  object 
 14  B_

In [39]:
notam.describe().round(3)

,NOTAM_REC_ID,LAUNCHES_REC_ID
count,1649078.00,0.0
mean,825311.65,NaN
std,476486.95,NaN
min,1.00,NaN
25%,412627.25,NaN
50%,825324.50,NaN
75%,1238009.75,NaN
max,1650468.00,NaN


In [40]:
notam.dtypes

NOTAM_REC_ID                int64
FNS_ID                     object
FILENAME                   object
NOTAM_ID                   object
NOTAM_TYPE                 object
RELATED_NOTAM_ID           object
SIMPLE_TEXT                object
LOC_ID_ACCOUNTABLE_ORG     object
NOTAM_NUMBER               object
RELATED_NOTAM_NUMBER       object
TEXT                       object
Q_CODE                     object
Q_CODE_INTERPRETATION      object
A_CODE                     object
B_CODE                     object
C_CODE                     object
D_CODE                     object
E_CODE                     object
F_CODE                     object
G_CODE                     object
CLASSIFICATION             object
POSSIBLE_NOTAM_ID          object
MIN_ALT                    object
MAX_ALT                    object
MIN_ALT_REF_TYPE           object
MAX_ALT_REF_TYPE           object
POSSIBLE_START_DATE        object
POSSIBLE_END_DATE          object
ISSUE_DATE                 object
CANCELED_DATE 

In [41]:
notam.isnull().sum()

NOTAM_REC_ID                    0
FNS_ID                          0
FILENAME                        0
NOTAM_ID                       29
NOTAM_TYPE                 221143
RELATED_NOTAM_ID          1280946
SIMPLE_TEXT                821116
LOC_ID_ACCOUNTABLE_ORG     863429
NOTAM_NUMBER               866592
RELATED_NOTAM_NUMBER      1523512
TEXT                         8066
Q_CODE                     640119
Q_CODE_INTERPRETATION      908595
A_CODE                     508043
B_CODE                     666797
C_CODE                     820617
D_CODE                    1479291
E_CODE                        448
F_CODE                     629677
G_CODE                     629900
CLASSIFICATION                 35
POSSIBLE_NOTAM_ID              30
MIN_ALT                    461494
MAX_ALT                    514100
MIN_ALT_REF_TYPE           597539
MAX_ALT_REF_TYPE           665667
POSSIBLE_START_DATE             3
POSSIBLE_END_DATE           22156
ISSUE_DATE                     31
CANCELED_DATE 

In [42]:
duplicates = notam.duplicated()
print(f'Number of duplicate rows = {duplicates}')

Number of duplicate rows = 0          False
1          False
2          False
3          False
4          False
           ...  
1649073    False
1649074    False
1649075    False
1649076    False
1649077    False
Length: 1649078, dtype: bool
